In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
import sys
 
# adding Folder_2 to the system path
sys.path.insert(0, '/home/fanuel-data/Projects/Week_11/Contract_Advisor_RAG/backend')

In [2]:
from langchain import OpenAI
from langchain.document_loaders import TextLoader , PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import NLTKTextSplitter
from langchain_community.document_loaders import Docx2txtLoader
from Evaluation import Evaluation
import pandas as pd
import nltk
nltk.download('punkt')

/home/fanuel-data/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/fanuel-
[nltk_data]     data/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
evaluation = Evaluation()

In [4]:
questions = [
    "Who are the parties to the Agreement and what are their defined names?",
    "What is the termination notice?",
    "What are the payments to the Advisor under the Agreement?",
    "Can the Agreement or any of its obligations be assigned?",
    "Who owns the IP?",
    "Is there a non-compete obligation to the Advisor?",
    "Can the Advisor charge for meal time?",
    "In which street does the Advisor live?",
    "Is the Advisor entitled to social benefits?",
    "What happens if the Advisor claims compensation based on employment relationship with the Company?"
]

ground_truths = [["Cloud Investments Ltd. (“Company”) and Jack Robinson (“Advisor”)"],
    ["According to section 4:14 days for convenience by both parties. The Company may terminate without notice if the Advisor refuses or cannot perform the Services or is in breach of any provision of this Agreement."],
    ["According to section 6: 1. Fees of $9 per hour up to a monthly limit of $1,500, 2. Workspace expense of $100 per month, 3. Other reasonable and actual expenses if approved by the company in writing and in advance."],
    ["Under section 1.1 the Advisor can’t assign any of his obligations without the prior written consent of the Company, 2. Under section 9  the Advisor may not assign the Agreement and the Company may assign it, 3 Under section 9 of the Undertaking the Company may assign the Undertaking."],
    ["According to section 4 of the Undertaking (Appendix A), Any Work Product, upon creation, shall be fully and exclusively owned by the Company."],
    ["Yes. During the term of engagement with the Company and for a period of 12 months thereafter."],
    ["No. See Section 6.1, Billable Hour doesn’t include meals or travel time. "],
    ["1 Rabin st, Tel Aviv, Israel "],
    ["No. According to section 8 of the Agreement, the Advisor is an independent consultant and shall not be entitled to any overtime pay, insurance, paid vacation, severance payments or similar fringe or employment benefits from the Company."],
    ["If the Advisor is determined to be an employee of the Company by a governmental authority, payments to the Advisor will be retroactively reduced so that 60% constitutes salary payments and 40% constitutes payment for statutory rights and benefits. The Company may offset any amounts due to the Advisor from any amounts payable under the Agreement. The Advisor must indemnify the Company for any losses or expenses incurred if an employer/employee relationship is determined to exist."]
]

# Load data

In [5]:
loader = Docx2txtLoader("../data/robinson_advisory.docx")
documents = loader.load()

# Chunk documents

In [6]:
text_splitter = CharacterTextSplitter(separator = "\n\n",
    chunk_size = 512,
    chunk_overlap  = 20)
texts  = text_splitter.split_documents(documents)
texts

Created a chunk of size 525, which is longer than the specified 512
Created a chunk of size 793, which is longer than the specified 512
Created a chunk of size 655, which is longer than the specified 512
Created a chunk of size 550, which is longer than the specified 512
Created a chunk of size 1330, which is longer than the specified 512
Created a chunk of size 692, which is longer than the specified 512
Created a chunk of size 877, which is longer than the specified 512
Created a chunk of size 534, which is longer than the specified 512
Created a chunk of size 616, which is longer than the specified 512
Created a chunk of size 538, which is longer than the specified 512
Created a chunk of size 705, which is longer than the specified 512


[Document(page_content='- 2-\n\nADVISORY SERVICES AGREEMENT\n\nThis Advisory Services Agreement is entered into as of June 15th, 2023 (the “Effective Date”), by and between Cloud Investments Ltd., ID 51-426526-3, an Israeli company (the "Company"), and Mr. Jack Robinson, Passport Number 780055578, residing at 1 Rabin st, Tel Aviv, Israel, Email: jackrobinson@gmail.com ("Advisor").', metadata={'source': '../data/robinson_advisory.docx'}),
 Document(page_content='Whereas,\tAdvisor has expertise and/or knowledge and/or relationships, which are relevant to the Company’s business and the Company has asked Advisor to provide it with certain Advisory services, as described in this Agreement; and\n\nWhereas, \tAdvisor has agreed to provide the Company with such services, subject to the terms set forth in this Agreement.\n\nNOW THEREFORE THE PARTIES AGREE AS FOLLOWS:\n\nServices:', metadata={'source': '../data/robinson_advisory.docx'}),
 Document(page_content='Services:  \n\nAdvisor shall provi

# Vectore Store

In [7]:
embeddings = OpenAIEmbeddings()
store = Chroma.from_documents(texts,embeddings, collection_name="contract")

/home/fanuel-data/miniconda3/envs/dapp/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


# Get Retriever

In [8]:
llm = OpenAI(temperature=0)
retriever=store.as_retriever()
rag_chain = RetrievalQA.from_chain_type(llm,retriever=retriever)

/home/fanuel-data/miniconda3/envs/dapp/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [9]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [10]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

In [11]:
question = "In which street does the Advisor live?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

I don't know.


In [12]:
print(result["context"])

[Document(page_content='Jurisdiction: This Undertaking shall be governed by and construed in accordance with the laws of the State of Israel, without giving effect to its laws pertaining to conflict of laws.\xa0 The Advisor agrees that any and all disputes in connection with this Undertaking shall be submitted to the exclusive jurisdiction of the competent courts in the city of Tel Aviv-Yafo, Israel.', metadata={'source': '../data/robinson_advisory.docx'}), Document(page_content='In performing the Services, Advisor shall devote his time, know-how, expertise, talent, experience and best efforts and attention to the provision of the Services, as shall be reasonably required by the Company. Advisor shall render the Services in a diligent manner and at a high professional standard for the benefit of the Company and shall perform and discharge well and faithfully, with devotion, honesty and fidelity, his obligations hereunder.', metadata={'source': '../data/robinson_advisory.docx'}), Docume

# Evaluate with RAGAs

In [13]:
dataset = evaluation.get_dataset(questions,ground_truths,retrieval_augmented_qa_chain,retriever,1)
dataset["contexts"]

[['Term: The term of this Agreement shall commence on the Effective Date and shall continue until terminated in accordance with the provisions herein (the "Term").',
  'Governing Law and Jurisdiction:  This Agreement shall be governed by the laws of the State of Israel, without giving effect to the rules respecting conflicts of laws. The parties consent to the exclusive jurisdiction and venue of Tel Aviv courts for any lawsuit filed arising from or relating to this Agreement.',
  'Whereas,\tAdvisor has expertise and/or knowledge and/or relationships, which are relevant to the Company’s business and the Company has asked Advisor to provide it with certain Advisory services, as described in this Agreement; and\n\nWhereas, \tAdvisor has agreed to provide the Company with such services, subject to the terms set forth in this Agreement.\n\nNOW THEREFORE THE PARTIES AGREE AS FOLLOWS:\n\nServices:',
  'Entire Agreement; No Waiver or Assignment: This Agreement together with the Exhibits, which

In [14]:

ddf = dataset.to_pandas()
ddf

,question,answer,contexts,ground_truths
0,Who are the parties to the Agreement and what ...,I don't know.,[Term: The term of this Agreement shall commen...,[Cloud Investments Ltd. (“Company”) and Jack R...
1,What is the termination notice?,The termination notice is fourteen (14) days' ...,"[Termination: Either party, at any given time,...",[According to section 4:14 days for convenienc...
2,What are the payments to the Advisor under the...,The payments to the Advisor under the Agreemen...,[As full and sole consideration for the Servic...,[According to section 6: 1. Fees of $9 per hou...
3,Can the Agreement or any of its obligations be...,"No, the Agreement or any of its obligations ca...",[Entire Agreement; No Waiver or Assignment: Th...,[Under section 1.1 the Advisor can’t assign an...
4,Who owns the IP?,The Company owns the IP.,"[IP: Any Work Product, upon creation, shall be...",[According to section 4 of the Undertaking (Ap...
5,Is there a non-compete obligation to the Advisor?,Yes.,[Non-Compete: During the term of engagement wi...,[Yes. During the term of engagement with the C...
6,Can the Advisor charge for meal time?,"No, the Advisor cannot charge for meal time.",[As full and sole consideration for the Servic...,"[No. See Section 6.1, Billable Hour doesn’t in..."
7,In which street does the Advisor live?,I don't know.,[Jurisdiction: This Undertaking shall be gover...,"[1 Rabin st, Tel Aviv, Israel ]"
8,Is the Advisor entitled to social benefits?,"No, the Advisor is not entitled to social bene...",[Advisor shall be solely responsible for any i...,"[No. According to section 8 of the Agreement, ..."
9,What happens if the Advisor claims compensatio...,If the Advisor claims compensation based on an...,[Relationship of the Parties; Indemnification:...,[If the Advisor is determined to be an employe...


In [15]:
results = evaluation.evaluate_RAG(dataset)
df = results.to_pandas()


evaluating with [context_precision]


100%|██████████| 1/1 [00:02<00:00,  2.96s/it]


evaluating with [context_recall]


100%|██████████| 1/1 [00:05<00:00,  5.69s/it]


evaluating with [faithfulness]


100%|██████████| 1/1 [00:10<00:00, 10.74s/it]


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:13<00:00, 13.04s/it]


In [16]:
results['context_precision']

0.5833333332791667

In [17]:
df

,question,answer,contexts,ground_truths,context_precision,context_recall,faithfulness,answer_relevancy
0,Who are the parties to the Agreement and what ...,I don't know.,[Term: The term of this Agreement shall commen...,[Cloud Investments Ltd. (“Company”) and Jack R...,0.000000,0.5,0.0,0.000000
1,What is the termination notice?,The termination notice is fourteen (14) days' ...,"[Termination: Either party, at any given time,...",[According to section 4:14 days for convenienc...,0.000000,0.5,1.0,0.880090
2,What are the payments to the Advisor under the...,The payments to the Advisor under the Agreemen...,[As full and sole consideration for the Servic...,[According to section 6: 1. Fees of $9 per hou...,1.000000,1.0,1.0,0.982625
3,Can the Agreement or any of its obligations be...,"No, the Agreement or any of its obligations ca...",[Entire Agreement; No Waiver or Assignment: Th...,[Under section 1.1 the Advisor can’t assign an...,1.000000,1.0,1.0,0.956353
4,Who owns the IP?,The Company owns the IP.,"[IP: Any Work Product, upon creation, shall be...",[According to section 4 of the Undertaking (Ap...,1.000000,1.0,1.0,1.000000
5,Is there a non-compete obligation to the Advisor?,Yes.,[Non-Compete: During the term of engagement wi...,[Yes. During the term of engagement with the C...,0.833333,1.0,1.0,0.000000
6,Can the Advisor charge for meal time?,"No, the Advisor cannot charge for meal time.",[As full and sole consideration for the Servic...,"[No. See Section 6.1, Billable Hour doesn’t in...",0.000000,1.0,1.0,1.000000
7,In which street does the Advisor live?,I don't know.,[Jurisdiction: This Undertaking shall be gover...,"[1 Rabin st, Tel Aviv, Israel ]",0.000000,0.0,0.0,0.834685
8,Is the Advisor entitled to social benefits?,"No, the Advisor is not entitled to social bene...",[Advisor shall be solely responsible for any i...,"[No. According to section 8 of the Agreement, ...",1.000000,1.0,1.0,1.000000
9,What happens if the Advisor claims compensatio...,If the Advisor claims compensation based on an...,[Relationship of the Parties; Indemnification:...,[If the Advisor is determined to be an employe...,1.000000,1.0,1.0,0.885373


In [18]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

In [23]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [24]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

In [25]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [28]:
response = retrieval_chain.invoke({"input": "Who are the parties to the Agreement and what are their defined names?"})

In [29]:
print(response["answer"])

The parties to the Agreement are the Advisor and the Company. Their defined names are not explicitly mentioned in the provided context.


In [ ]:
dataset_r = evaluation.get_dataset(questions,ground_truths,retrieval_chain,retriever,2)